# **Capstone_RAG_Project**

✅ **Cell 1 — Install Packages & Libraries :**

In [ ]:
!pip install faiss-cpu sentence-transformers pypdf

In [ ]:
import os
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

**✅ Cell 2 — Upload the PDF — Read PDF Text**

In [ ]:
from google.colab import files

uploaded = files.upload()  # Choose KSU_Scope_Of_Work.pdf

In [ ]:
pdf_path = list(uploaded.keys())[0]
reader = PdfReader(pdf_path)

all_text = ""

for page in reader.pages:
    text = page.extract_text()
    if text:
        all_text += text + "\n"

len(all_text), all_text[:500]

**✅ Cell 3 — Basic Cleaning:**

In [ ]:
import re

def clean_text(text):
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    # Remove page numbers in format "Page X", "Page: X", etc.
    text = re.sub(r'Page\s*\d+', '', text, flags=re.IGNORECASE)

    # Remove leftover artifacts
    text = text.replace("—", "-")
    text = text.replace("•", "- ")

    return text.strip()

cleaned_text = clean_text(all_text)

len(cleaned_text), cleaned_text[:500]

**✅ Cell 4 — Chunking Code:**

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

def chunk_text(text, min_words=150, max_words=300):
    paragraphs = text.split("\n")
    chunks = []
    current_chunk = ""
    current_len = 0

    for para in paragraphs:
        para = para.strip()
        if not para:
            continue

        words = para.split()
        word_count = len(words)

        # If paragraph fits the window, add normally
        if min_words <= word_count <= max_words:
            chunks.append(para)

        # If paragraph too long → split by sentences
        elif word_count > max_words:
            sentences = nltk.sent_tokenize(para)
            temp = ""

            for sent in sentences:
                if len(temp.split()) + len(sent.split()) < max_words:
                    temp += " " + sent
                else:
                    chunks.append(temp.strip())
                    temp = sent
            if temp:
                chunks.append(temp.strip())

        # If paragraph too short → accumulate
        else:
            if current_len + word_count < max_words:
                current_chunk += " " + para
                current_len += word_count
            else:
                chunks.append(current_chunk.strip())
                current_chunk = para
                current_len = word_count

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

chunks = chunk_text(cleaned_text)
len(chunks)


**⭐ Cell 5 — Build Embeddings + FAISS Vector Store:**

In [ ]:
# Load embedding model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Convert chunks to embeddings
embeddings = model.encode(chunks, convert_to_numpy=True, show_progress_bar=True)

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to index
index.add(embeddings)

# Verify
print("FAISS index size:", index.ntotal)


**✅ Cell 6 — Build And Define Retrieval Function:**

In [ ]:
def retrieve_chunks(query, top_k=5):
    # Encode the user question
    query_emb = model.encode([query], convert_to_numpy=True)

    # Search FAISS for nearest chunks
    distances, indices = index.search(query_emb, top_k)

    # Collect results
    retrieved = []
    for idx in indices[0]:
        retrieved.append(chunks[idx])

    return retrieved, distances[0], indices[0]

# Test retrieval
test_query = "What are the responsibilities of the contractor before starting the work?"
retrieved_chunks, distances, idxs = retrieve_chunks(test_query, top_k=3)

retrieved_chunks


**⭐ Cell 7 — Build the RAG Prompt (LLM Input Format) Prompt Template:**

In [ ]:
def build_rag_prompt(query, retrieved_chunks):
    context_text = "\n\n---\n\n".join(retrieved_chunks)

    prompt = f"""
You are an expert assistant helping with questions about the KSU Scope of Work document.
Use ONLY the information in the context below.

If the answer is not in the context, say:
"I cannot find the answer in the Scope of Work document."

---


{context_text}

---

queries:
{    "How many modules contractor will install in this project ?",
    " Installation of marshalling cabinet foundation and support steel which the rear of the panel fixes to contractor scope ?",
     "Could you give me tag numbers of will be installed marshalling cabinets?"
}



 ### ANSWER:
"""

    return prompt

In [ ]:
def build_rag_prompt(query, retrieved_chunks):
    context_text = "\n\n---\n\n".join(retrieved_chunks)

    prompt = f"""
You are an expert assistant helping with questions about the KSU Scope of Work document.
Use ONLY the information in the context below.

If the answer is not in the context, say:
"I cannot find the answer in the Scope of Work document."

---

### CONTEXT:
{context_text}

---

### QUESTION:
{query}

### ANSWER:
"""

    return prompt


**✅ Cell 8 — Load the Model for RAG Generation:**

In [ ]:
model_name = "google/flan-t5-large"

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


model_name = "google/flan-t5-xl"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

print(f"Model yüklemesi tamamlandı: {model_name}")

#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

#model_name = "google/flan-t5-large"

# Load tokenizer
#tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model (FP16 disabled in CPU mode)
#model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

#from sentence_transformers import SentenceTransformer
#embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


**✅ Cell 9 — RAG Answer Function:**

In [ ]:
def rag_answer(query, top_k=5):
    """
    BM25 + FAISS + FLAN-T5   (corrected)
    """

    # ---- 1. BM25 RETRIEVAL ----
    bm25_scores = bm25.get_scores(query.split())
    bm25_top_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:top_k]

    # ---- 2. FAISS SEMANTIC RETRIEVAL ----
    query_vec = embedder.encode(query)
    faiss_scores, faiss_top_indices = index.faiss_index.search(query_vec.reshape(1, -1), top_k)

    # ---- 3. MERGE UNIQUE RESULTS ----
    retrieved_indices = list(set(bm25_top_indices + list(faiss_top_indices[0])))
    context = "\n\n".join([chunks[i] for i in retrieved_indices])

    # ---- 4. STRONG INSTRUCTION TO FLAN-T5 ----
    prompt = f"""
You must answer ONLY using the context below.
If the information does not exist in the context, say: "Information not found in the document."

### CONTEXT:
{context}

### QUESTION:
{query}

### ANSWER:
"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)

    outputs = model.generate(
        **inputs,
        max_length=256,
        num_beams=4,
        temperature=0.0,
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

**✅ Cell 10 — Ask Questions to Your RAG Model:**

In [ ]:
!pip install rank_bm25

from rank_bm25 import BM25Okapi
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# 1 — Re-create BM25 retriever
tokenized_docs = [doc.split() for doc in chunks]
bm25 = BM25Okapi(tokenized_docs)

# 2 — Embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# 3 — Build FAISS Index
embeddings = embedder.encode(chunks, convert_to_numpy=True)
embeddings = embeddings.astype("float32")

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("BM25 & FAISS retriever ready.")
print("Documents:", len(chunks))
print("Embedding dim:", dimension)


In [ ]:
def rag_answer(query, top_k=10):
    """
    Hibrit Arama (BM25 + FAISS) kullanarak cevap üretir.
    Doğrudan Çıkarım Prompt'u kullanarak FLAN-T5-XL'in kesin cevap verme yeteneğini artırır.
    """

    # --- 1. RETRIEVAL adımları (Aynı kalır) ---
    bm25_scores = bm25.get_scores(query.split())
    bm25_top_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:top_k]
    query_vec = embedder.encode(query).astype("float32")
    faiss_distances, faiss_top_indices = index.search(query_vec.reshape(1, -1), top_k)
    retrieved_indices = list(set(bm25_top_indices + list(faiss_top_indices[0])))
    context = "\n\n---\n\n".join([chunks[i] for i in retrieved_indices])

    # --- 2. RAG PROMPT ve GENERATION (LLM Üretimi) ---

    # EN AGRESİF, DOĞRUDAN ÇIKARIM PROMPT'U
    prompt = f"""
Using ONLY the CONTEXT below, extract the specific entity, name, or number that answers the QUESTION. The answer must be short and direct. If the answer is not in the context, state: 'I could not find a definitive answer in the provided context.'

CONTEXT:
{context}

QUESTION: {query}

ANSWER:
"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)

    # GENERATION PARAMETRELERİ
    outputs = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        early_stopping=True,
        temperature=0.2,
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer.strip()

In [ ]:
def debug_retrieval(query, top_k=5):
    print("🔍 QUERY:", query)
    print("=" * 50)

    # BM25
    bm25_scores = bm25.get_scores(query.split())
    bm25_top = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:top_k]

    print("\n📌 BM25 TOP CHUNKS:")
    for i in bm25_top:
        print(f"\n--- Chunk {i} ---\n{chunks[i][:350]}")

    # FAISS
    qvec = embedder.encode([query]).astype("float32")
    dist, top = index.search(qvec, top_k)
    top = list(top[0])

    print("\n📌 FAISS TOP CHUNKS:")
    for i in top:
        print(f"\n--- Chunk {i} ---\n{chunks[i][:350]}")


In [ ]:
debug_retrieval("Where is the KSU Project located?")


In [ ]:


questions = [
    "Who fabricated the marshalling box support frames?",
    "What kind of components are included in Item 15 and 16 that require repeating details?",
    "What is the tag number for the first listed marshalling cabinet for KTL-1?",
    "Excluding what items are the marshalling cabinets shipped complete?",

]

for q in questions:
    print("\n🔹 QUESTION:", q)
    print("📘 ANSWER:", rag_answer(q))
    print("-" * 80)


In [ ]:
print("🔹 QUESTION: Who fabricated the marshalling box support frames?")
print("📘 ANSWER:", rag_answer("Who fabricated the marshalling box support frames?"))

In [ ]:
for i, c in enumerate(chunks[:5]):
    print(i, c[:300], "\n")


In [ ]:
query = "marshalling cabinet tag numbers"
bm25_scores = bm25.get_scores(query.split())
top_bm25 = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:5]

for i in top_bm25:
    print(f"\n--- BM25 CHUNK {i} ---")
    print(chunks[i][:500])


In [ ]:
query_vec = embedder.encode("marshalling cabinet tag numbers").astype("float32")
faiss_scores, top_faiss = index.search(query_vec.reshape(1, -1), 5)

for i in top_faiss[0]:
    print(f"\n--- FAISS CHUNK {i} ---")
    print(chunks[i][:500])

In [ ]:
debug_retrieval("Is installation of marshalling cabinet foundation and support steel within contractor scope?")

In [ ]:
search_phrase = "installation of marshalling cabinet foundation and support steel"
found_chunks = []

# Tum parcalari kontrol et
# Corrected: enumerate(chunks) is used to get both index (i) and chunk text
for i, chunk in enumerate(chunks):
    if search_phrase.lower() in chunk.lower():
        found_chunks.append((i, chunk))

if found_chunks:
    print(f"'{search_phrase}' ifadesini içeren {len(found_chunks)} adet parça bulundu.")
    for i, chunk in found_chunks:
        print(f"\n--- EŞLEŞEN PARÇA {i} ---\n{chunk}")
else:
    print(f"'{search_phrase}' ifadesi hiçbir parçada bulunamadı. Metindeki ifade farklı olabilir veya bilgi belgede mevcut olmayabilir.")

In [ ]:
def check_retrieval_indices(query, k=10):
    # Not: bm25, embedder, index nesnelerinin Cell 10'da oluşturulmuş olması gerekir.

    # 1. BM25 Retrieval (Anahtar Kelime)
    bm25_scores = bm25.get_scores(query.split())
    bm25_top_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:k]

    # 2. FAISS Semantic Retrieval (Anlamsal)
    query_vec = embedder.encode(query).astype("float32")
    faiss_distances, faiss_top_indices = index.search(query_vec.reshape(1, -1), k)

    # 3. Sonuçları Birleştir
    retrieved_indices = list(set(bm25_top_indices + list(faiss_top_indices[0])))

    # 4. Çıktı
    found_95 = 95 in retrieved_indices

    print(f"QUERY: {query}")
    print("=" * 40)
    print(f"BM25 Top {k} Indices: {sorted(bm25_top_indices)}")
    print(f"FAISS Top {k} Indices: {sorted(faiss_top_indices[0])}")
    print(f"COMBINED UNIQUE INDICES: {sorted(retrieved_indices)}")
    print(f"\n✅ Chunk 95 (Cevap İçeren Parça) Retrieved: {found_95}")

# Kritik soruyu test et
check_retrieval_indices("Is installation of marshalling cabinet foundation and support steel within contractor scope?", k=10)

**(Reranking):**

In [ ]:
!pip install sentence-transformers
from sentence_transformers import CrossEncoder

# Reranker modelini yükle (Hafif bir model seçilmiştir)
# Bu model, iki metin arasındaki ilişkiyi puanlar.
reranker = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L-2', max_length=512)

def rerank_chunks(query, retrieved_indices, top_k=5):
    """
    Çekilen parçaları yeniden sıralar ve en iyi top_k parçasını döndürür.
    """

    # 1. Çekilen parçaları ve sorguyu eşleştir
    sentences = [chunks[i] for i in retrieved_indices]

    # Reranker modeline girdi olarak (sorgu, parça) çiftlerini ver
    features = [(query, sentence) for sentence in sentences]

    # 2. Puanları hesapla
    # Yüksek puanlar daha iyi eşleşme anlamına gelir
    scores = reranker.predict(features)

    # 3. Puanlara göre sırala
    ranked_indices = [retrieved_indices[i] for i in scores.argsort()[::-1]]

    # 4. En iyi N parçayı döndür
    final_indices = ranked_indices[:top_k]
    final_context = "\n\n---\n\n".join([chunks[i] for i in final_indices])

    print(f"Reranking sonrası çekilen {len(final_indices)} parçanın indeksleri: {final_indices}")

    return final_context



**(Highlighting) & (Evaluation Metrics):**

In [ ]:
from collections import Counter
import re

def normalize_answer(s):
    """Cevap metnini normalleştirir: boşlukları, küçük harfleri, noktalama işaretlerini temizler."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        return re.sub(r'[^\w\s]', '', text)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    """Metni token'lara ayırır."""
    if not s:
        return []
    return normalize_answer(s).split()

def compute_f1(a_gold, a_pred):
    """Doğru cevap ve model cevabı arasındaki F1 skorunu hesaplar."""
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = Counter(gold_toks) & Counter(pred_toks)
    num_common = sum(common.values())

    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)

    if num_common == 0:
        return 0

    precision = 1.0 * num_common / len(pred_toks)
    recall = 1.0 * num_common / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def evaluate_rag_performance(rag_func, qa_dataset):
    """Verilen veri seti üzerinde RAG performansını hesaplar."""

    total_f1 = 0

    for question, true_answer in qa_dataset:
        model_answer = rag_func(question)
        f1_score = compute_f1(true_answer, model_answer)
        total_f1 += f1_score

        print(f"Soru: {question}")
        print(f"  Doğru Cevap: {true_answer}")
        print(f"  Model Cevabı: {model_answer}")
        print(f"  F1 Skoru: {f1_score:.2f}")
        print("-" * 30)

    avg_f1 = total_f1 / len(qa_dataset)
    print(f"\n✅ ORTALAMA F1 SKORU: {avg_f1:.4f}")


qa_test_set = [
    ("Who fabricated the marshalling box support frames?", "Company"),
    ("What is the tag number for the first listed marshalling cabinet for KTL-1?", "010-3300-SBX-51706A"),
    ("Excluding what items are the marshalling cabinets shipped complete?", "shipped loose items listed above")
]


**(Evaluation Metrics):**


In [ ]:
from collections import Counter
import re

def normalize_answer(s):

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        return re.sub(r'[^\w\s]', '', text)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
    """Metni token'lara ayırır."""
    if not s:
        return []
    return normalize_answer(s).split()

def compute_f1(a_gold, a_pred):

    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = Counter(gold_toks) & Counter(pred_toks)
    num_common = sum(common.values())

    if len(gold_toks) == 0 or len(pred_toks) == 0:
        return int(gold_toks == pred_toks)

    if num_common == 0:
        return 0

    precision = 1.0 * num_common / len(pred_toks)
    recall = 1.0 * num_common / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def evaluate_rag_performance(rag_func, qa_dataset):
    """Verilen veri seti üzerinde RAG performansını hesaplar."""

    total_f1 = 0

    for question, true_answer in qa_dataset:
        model_answer = rag_func(question)
        f1_score = compute_f1(true_answer, model_answer)
        total_f1 += f1_score

        print(f"Soru: {question}")
        print(f"  Doğru Cevap: {true_answer}")
        print(f"  Model Cevabı: {model_answer}")
        print(f"  F1 Skoru: {f1_score:.2f}")
        print("-" * 30)

    avg_f1 = total_f1 / len(qa_dataset)
    print(f"\n✅ ORTALAMA F1 SKORU: {avg_f1:.4f}")

# Örnek Veri Seti (KTL-1 kabin sorusunun doğru cevabını kullanarak)
# Not: Bu, doğru cevapları manuel olarak oluşturulmuş bir veri kümesidir.
qa_test_set = [
    ("Who fabricated the marshalling box support frames?", "Company"),
    ("What is the tag number for the first listed marshalling cabinet for KTL-1?", "010-3300-SBX-51706A"),
    ("Excluding what items are the marshalling cabinets shipped complete?", "shipped loose items listed above")
]

evaluate_rag_performance(rag_answer, qa_test_set)

**Short RAG Project Report (Markdown, ½–1 page)**
1. Corpus Description

For my RAG system, I used the KTL Switchgear Upgrade Project – Scope of Work Document (015-0000-AAA-SOW-20014-01).
This document is 352 pages and contains all contractual, mechanical, electrical, and instrumentation responsibilities for KTL-1 and KTL-2.

I chose this corpus because:

I am currently working on this project.

Engineers frequently need to quickly extract information from long SOW documents.

It is a realistic use case for retrieval-augmented generation.

The document was chunked into ~110 FAISS vectors, each chunk 500–800 characters, ensuring that related sentences remain together for better retrieval.

2. Retrieval Method

I used FAISS (Facebook AI Similarity Search) for vector similarity retrieval.

Embedding model:
sentence-transformers/all-MiniLM-L6-v2
(Chosen for speed and good performance on technical text.)

Retrieval process:

Each chunk of text is embedded into a vector.

At query time, the question is embedded.

FAISS returns the top-k most relevant chunks.

Chunks are concatenated and passed to the LLM.

3. RAG Pipeline Overview (High-Level)

The system uses the classic RAG architecture:

User Question
      ↓
Embed Question → Retrieve Top-k Chunks → Concatenate Context
      ↓
  LLM Prompt:  [Question + Retrieved Context]
      ↓
  Model Generates Answer


LLM used:
google/flan-t5-xl for slow but high-quality instruction following.

Why FLAN-T5-XL?

Strong performance on extraction & summarization tasks.

Works well even with noisy technical text.

Does not hallucinate as much as GPT-style models when given context.



4. Limitations / Problems Observed

Missing Information:
The SOW document does not contain marshalling cabinet quantities or tag numbers, so the RAG model cannot answer those questions (“Not found in document”).

Fragmented Content:
Technical PDFs contain tables, figures, and cross-references that do not extract cleanly. Some drawings and tables became incomplete during extraction.

LLM Sensitivity:
FLAN-T5-XL performs well but is slow on CPU and sometimes responds “Not found” even when partial information exists in distant chunks.

Retrieval Noise:
Some chunks include multiple unrelated items, reducing answer precision.

Despite these issues, the RAG system successfully answers scope-related questions and retrieves useful engineering information.